# Installing all required libraries

In [1]:
import heapq
import time
import matplotlib.pyplot as plt
import tkinter
import numpy
import plotly

In [2]:
# checking whether assignment is valid
def check_valid_assignment(assignment, constraints):
    for constraint in constraints:
        if not constraint(assignment):
            return False
    return True
# forward checking function
def forward_check(assignment, variables, domains, constraints):
    for variable in variables:
        if variable not in assignment:
            for value in list(domains[variable]):
                assignment[variable] = value
                if not check_valid_assignment(assignment, constraints):
                    domains[variable].remove(value)
            assignment[variable] = None
# forward checking search function
def forward_check_search(assignment, variables, domains, constraints):
    if len(assignment) == len(variables):
        return assignment

    variable = [var for var in variables if var not in assignment]

    # check if all variables are assigned
    if not variable:
        return assignment

    variable = variable[0]

    for value in domains[variable]:
        assignment[variable] = value

        forward_check(assignment, variables, domains, constraints)

        if check_valid_assignment(assignment, constraints):
            result = forward_check_search(assignment, variables, domains, constraints)

            if result is not None:
                return result

        assignment[variable] = None

    return None

# CSP scenarios
scenario_1_vars = ['Juan', 'Jane', 'Jim']
scenario_2_vars = ['Anita', 'Peter','Jim', 'Jane']

scenario_1_domains = {
    'Juan': ['Web Designer', 'AI Engineer'],
    'Jim': ['AI Engineer', 'Systems Engineer'],
    'Jane': ['Python Programmer', 'Database Admin']
}

scenario_2_domains = {
    'Jim': ['AI Engineer', 'Systems Engineer'],
    'Jane': ['Python Programmer', 'Database Admin'],
    'Anita': ['Web Designer', 'AI Engineer'],
    'Peter': ['Python Programmer', 'AI Engineer']
}

# assigning Ciara as the first Python Programmer
first_assignment_scenario_1 = {'Ciara': 'Python Programmer'}

# defining the constraint that each person can have at most two roles
def validate_roles_limit(assignment):
    # handling uninitialized roles
    role_count = {role: 0 for role in ['Python Programmer', 'AI Engineer', 'Web Designer', 'Database Admin', 'Systems Engineer']}

    for roles in assignment.values():
        for role in roles:
            # checking if the role is in the role_count dictionary
            if role in role_count:
                role_count[role] += 1
                if role_count[role] > 2:
                    return False
            else:
                return False
    return True

constraints = [validate_roles_limit]

# determining who to hire in  Scenario 1
scenario_1_sol = forward_check_search(first_assignment_scenario_1, scenario_1_vars, scenario_1_domains, constraints.copy())

# determining who to hire in  Scenario 2
scenario_2_sol = forward_check_search({}, scenario_2_vars, scenario_2_domains, constraints.copy())

print("Ciara's Company should hire:\n", scenario_1_vars)
print("\nCiara & Juan Partnership should hire:\n", scenario_2_vars)

Ciara's Company should hire:
 ['Juan', 'Jane', 'Jim']

Ciara & Juan Partnership should hire:
 ['Anita', 'Peter', 'Jim', 'Jane']
